In [364]:
# Load Bridge helper classes
%run helpers/bridge.ipynb
%run helpers/load-data.ipynb
%run helpers/train-eval-utils.ipynb

In [ ]:
# Loads & precomputes all the data, takes about 5min
data = load_deals()

# Helpers

In [226]:
class Strategy(object):
    
    def __init__(self, function):
        # Takes as an input a function from (Hand, Bidding) -> bid
        self.fn = function
        
    def step(self, hand, bidding):
        # In case the bid is invalid, automatically substitute for PASS (idk)
        bid = self.fn(hand, bidding)
        if isinstance(bid, str):
            bid = valid_bids.index(bid)
        if 0 < bid <= len(contracts) and bid > bidding.last_bid():
            return bidding.add(self.fn(hand, bidding))
        return bidding.add(0)

In [121]:
def gen_sequence(deal, s1, s2=Strategy(pass_function), s3=None, s4=None):
    seq = Bidding()
    if s3 is None:
        s3 = s1
    if s4 is None:
        s4 = s2
    sgs = [s1, s2, s3, s4]
    while not seq.done:
        cur_pl = len(seq) % 4
        seq = sgs[cur_pl].step(deal.hands[cur_pl], seq)
    return seq

In [322]:
def eval_strategy(list_of_deals, strategy, enemy_s=Strategy(pass_function), strategyS=None,
                  with_opps=True):
    if not isinstance(list_of_deals, list):
        list_of_deals = [list_of_deals]
    rt = 0
    for d in list_of_deals:
        seq = gen_sequence(d, strategy, enemy_s, strategyS)
        res, side = seq.result()
        rt += d.ev(res, side, with_opps=with_opps)
    return rt / len(list_of_deals)

# Examples

In [ ]:
# Full knowledge

## One hand places the contract

In [219]:
def const_function(contr):
    def fn(hand, bidding):
        return contr
    return fn

In [ ]:
def pass_function(hand, bidding):
    return const_function('PASS')

In [230]:
def hcp_function(start=15, step=3):
    def fn(hand, bidding):
        ft = hand.features()
        if ft['hcp'] >= start:
            lev = min(7, (ft['hcp'] - start) / step + 1)
            return str(int(lev)) + 'NT'
        return 'PASS'
    return fn

In [210]:
def suit_function(start=15, step=3):
    def fn(hand, bidding):
        ft = hand.features()
        if ft['hcp'] >= start:
            lev = min(7, (ft['hcp'] - start) / step + 1)
            strain = 'NT'
            bst_len = 4
            for s in suits:
                if ft[s + '_len'] >= bst_len:
                    strain = s
                    bst_len = ft[s + '_len']
            return str(int(lev)) + strain
            #return const_function(str(int(lev)) + strain)(hand, bidding)
        return 'PASS'
    return fn

In [ ]:
# Try to make a NN learn the best contract with_opps

In [358]:
X = [d.hands[0].feature_array() for d in data[:10 ** 4]] #only N hand

# Winners in a particular strain played from N
y = [d.precompute().best_contract(with_opps=True) for d in data[:10 ** 4]]

In [362]:
y[500:]

['7s',
 '3s',
 '3s',
 '4s',
 '4NT',
 '1s',
 '4h',
 '5s',
 '5NT',
 '3d',
 '3d',
 '3NT',
 '6h',
 '4d',
 '7s',
 '6d',
 '2c',
 '3c',
 '4h',
 '4NT',
 '7NT',
 '5h',
 '4d',
 '3NT',
 '2NT',
 '5NT',
 '3d',
 '3s',
 '4h',
 '5s',
 '2c',
 '6d',
 '2d',
 '6d',
 '4h',
 '7s',
 '5d',
 '4s',
 '3s',
 '5h',
 '6NT',
 '4h',
 '5h',
 '4NT',
 '5h',
 '3NT',
 '2NT',
 '3NT',
 '4d',
 '7NT',
 '4NT',
 '3h',
 '3NT',
 '5NT',
 '7s',
 '6NT',
 '2h',
 '3h',
 '3NT',
 '5c',
 '2h',
 '1s',
 '5h',
 '3h',
 '3s',
 '4h',
 '4NT',
 '3s',
 '5h',
 '2s',
 '6NT',
 '3NT',
 '4c',
 '3c',
 '3h',
 '4s',
 '3h',
 '4h',
 '3d',
 '2d',
 '3d',
 '3NT',
 '3h',
 '3NT',
 '5NT',
 '3c',
 '3h',
 '5NT',
 '4c',
 '4h',
 '7s',
 '6NT',
 '3s',
 '4h',
 '6NT',
 '4c',
 '4NT',
 '5h',
 '4d',
 '5d',
 '4c',
 '3d',
 '6d',
 '6s',
 '4h',
 '3NT',
 '2c',
 '5d',
 '3h',
 '2NT',
 '2s',
 '3h',
 '5c',
 '1s',
 '4s',
 '4s',
 '6h',
 '3h',
 '4NT',
 '5d',
 '3c',
 '5d',
 '1s',
 '4h',
 '3NT',
 '4NT',
 '1NT',
 '2d',
 '7s',
 '6d',
 '4h',
 '4h',
 '5s',
 '4s',
 '2NT',
 '7NT',
 '2d',
 '4N

## One hand describes, other places

In [ ]:
def base_function(hand, bidding):
    # if other hand has not described
    
    # if other hand has described

# TO RUN TONIGHT

In [ ]:
# Running time: 1hr
# Finds best values for bs, step for the hcp function
bsf = -100
for start in range(65, 90):
    for step in range(25, 50):
        if bsf < eval_strategy(data, Strategy(hcp_function(start / 5, step / 10))):
            bs, bstep = start / 5, step / 10

print (bs, bstep)

In [ ]:
def suit_function(start=15, step=3):
    def fn(hand, bidding):
        if hand.features()['hcp'] >= start:
            lev = min(7, (hand.features()['hcp'] - start) / step + 1)
            strain = 'NT'
            bst_len = 5
            for s in suits:
                if hand.features()[s + '_len'] >= bst_len:
                    strain = s
                    bst_len = hand.features()[s + '_len']
            return const_function(str(int(lev)) + strain)(hand, bidding)
        return 'PASS'
    return fn

In [ ]:
# Running time: 3hr
# Finds best values for bs, step for the SUIT fn with 5
bsf = -100
for start in range(30, 70):
    for step in range(15, ):
        if bsf < eval_strategy(data, Strategy(suit_function(start / 5, step / 10))):
            bs, bstep = start / 5, step / 10

print (bs, bstep)

In [ ]:
def suit_function(start=15, step=3):
    def fn(hand, bidding):
        if hand.features()['hcp'] >= start:
            lev = min(7, (hand.features()['hcp'] - start) / step + 1)
            strain = 'NT'
            bst_len = 5
            for s in suits:
                if hand.features()[s + '_len'] >= bst_len:
                    strain = s
                    bst_len = hand.features()[s + '_len']
            return const_function(str(int(lev)) + strain)(hand, bidding)
        return 'PASS'
    return fn

In [ ]:
# Running time: 3hr
# Finds best values for bs, step for the SUIT fn with 4
bsf = -100
for start in range(30, 70):
    for step in range(15, 50):
        if bsf < eval_strategy(data, Strategy(suit_function(start / 5, step / 10))):
            bs, bstep = start / 5, step / 10

print (bs, bstep)

In [328]:
eval_strategy(data[:5000], Strategy(pass_function), with_opps=True)

-4.458

In [329]:
eval_strategy(data[:5000], Strategy(hcp_function()), with_opps=True)

-4.233

In [330]:
eval_strategy(data[:5000], Strategy(suit_function()), with_opps=True)

-4.176